d1_ic50_small

In [8]:
# imports:

# standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# library used for functional enrichment analysis
from func_e.FUNC_E import FUNC_E  # a method that can do funtional enrichment analysis!
import func_e.vocabs.all as vocabs
# following code is necessary as some functions used in the funcE class produce future warnings but i can't fix it as it is not my code
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# importing the custom classes i built
from matrix_class import ProteinMatrix # ppi matrix 
from cluster_class import AllClusters # dictionary to hold all clusters (in form number of cluster : list of proteins in that cluster)
from degreelist_class import DegreeList # creates a list of all proteins in order of their degree

# helper functions for setting up program
from recipe_utils import initialize_matrix_clusters_degreelist
from recipe_utils import find_clusters_and_proteins_together

# helper functions for functional enrichment
from recipe_utils import print_querylist_of_clusters_to_file
from recipe_utils import print_protein_background_to_file
from recipe_utils import create_term_mapping_list
from recipe_utils import get_initialized_fe

 **must change filenames:**

In [9]:
interactions_file = "data/interactions/1_ppi_string_v2.txt"
clusters_file = "data/d1_ic50_small/d1_ic50_small.json"

In [10]:
matrix, clusters, degreelist = initialize_matrix_clusters_degreelist(interactions_filepath=interactions_file, clusters_filepath=clusters_file)

*may want to change parameters below:*

In [11]:
sqrt_qualifying_clusters, sqrt_qualifying_proteins = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=0, cluster_constant=2, use_sqrt=True, protein_ratio=1, protein_constant=0)

In [12]:
sqrt_qualifying_proteins

{101: ['WDR64', 'PSMD8', 'PSMD2'],
 129: ['SMAD2'],
 250: ['PIGA'],
 263: ['CTSD'],
 317: ['MPP6'],
 337: ['CCNK'],
 341: ['IDH3B'],
 356: ['PAX4'],
 431: ['WNT4'],
 515: ['TSPO'],
 523: ['SLC30A10'],
 533: ['CCNE1']}

In [13]:
# from recipe_utils import top_n_proteins

# three_qualifying_proteins = top_n_proteins(sqrt_qualifying_proteins, n=3) 
# three_qualifying_proteins

In [14]:
# from recipe_utils import get_cluster_connectivity

# original_cluster_connectivity = get_cluster_connectivity(matrix, degreelist, clusters, {}, sort_it=False)
# new_cluster_connectivity = get_cluster_connectivity(matrix, degreelist, clusters, added_proteins=sqrt_qualifying_proteins, sort_it=False)

# from recipe_utils import calculate_connectivity_difference

# difference = calculate_connectivity_difference(original_cluster_connectivity, new_cluster_connectivity, sort_it=True)
# difference

### Functional Enrichment Analysis

In [15]:
# create_term_mapping_list("data/go_labels/dream3_go.tsv", "data/go_labels/d3_term_mapping.tsv")

**will need to the directory below**

In [16]:
directory = "data/d1_ic50_small/"
term_mapping_filepath = "data/go_labels/d1_term_mapping.tsv"

In [17]:
background_proteins_filepath = directory + "background_proteinlist.txt"
og_query_filepath = directory + "original_clusters.txt"
sqrt_query_filepath = directory + "sqrt_clusters.txt"
# three_query_filepath = directory + "three_prot_clusters.txt"

*print to files*

In [18]:
# use functions to print info to files: (annoying, but need to get info into format used for funcE package)
# 1) a background protein list to be used for functional enrichment analysis
print_protein_background_to_file(matrix, background_proteins_filepath) 
# 3) not a file, but need to establish a background of all GO terms! (to be used for functional enrichment analysis)
background_go_terms = vocabs.getTerms(['GO'])

# continue printing info to files to be used in functional enrichment analysis:
# 4) print original clusters to a file
print_querylist_of_clusters_to_file(clusters, clusters.get_all_cluster_labels(), og_query_filepath)
# 5) print updated clusters to a file (just append the function to include the dictionary of qualifying proteins)
print_querylist_of_clusters_to_file(clusters, clusters.get_all_cluster_labels(), sqrt_query_filepath, sqrt_qualifying_proteins)
# print_querylist_of_clusters_to_file(clusters, clusters.get_all_cluster_labels(), three_query_filepath, three_qualifying_proteins)

*RUN Functional Enrichment:*

In [19]:
original_fe = get_initialized_fe(background_proteins_filepath, term_mapping_filepath, termlist = background_go_terms)
original_fe.importFiles({'query': og_query_filepath })
original_fe.run(cluster=False)

sqrt_fe = get_initialized_fe(background_proteins_filepath, term_mapping_filepath, termlist = background_go_terms)
sqrt_fe.importFiles({'query': sqrt_query_filepath })
sqrt_fe.run(cluster=False)

# three_fe = get_initialized_fe(background_proteins_filepath, term_mapping_filepath, termlist = background_go_terms)
# three_fe.importFiles({'query': three_query_filepath })
# three_fe.run(cluster=False)

In [20]:
print(f"::::::::::::::::::::::::::SQRT::::::::::::::::::::::::::\n")
# print data by the numbers!
print(f"number of clusters that were updated with proteins: {len(sqrt_qualifying_proteins.keys())}\n")

# clusters that are functionally enriched
print(f"number of functionally enriched clusters (modules) out of {clusters.get_num_clusters()} original clusters: {original_fe.enrichment['Module'].nunique()}")
print(f"number of functionally enriched clusters after addition of all proteins: {sqrt_fe.enrichment['Module'].nunique()}")
print(f"")


#significant P-values!!!
print(f"number of significant pvals in original clusters before updating: {original_fe.enrichment['Fishers_pvalue'].count()}")
print(f"number of significant pvals in clusters with all proteins added: {sqrt_fe.enrichment['Fishers_pvalue'].count()}")


print(f"")
print(f"")

::::::::::::::::::::::::::SQRT::::::::::::::::::::::::::

number of clusters that were updated with proteins: 12

number of functionally enriched clusters (modules) out of 544 original clusters: 483
number of functionally enriched clusters after addition of all proteins: 483

number of significant pvals in original clusters before updating: 1561
number of significant pvals in clusters with all proteins added: 1571




In [23]:
def top_n_proteins(
        qualifying_proteins:dict,
        n:int # max # of proteins to return
):
    n_proteins = dict()
    for key in qualifying_proteins:
        n_proteins[key] = qualifying_proteins[key][0:n]
    
    return n_proteins

In [24]:
# from recipe_utils import top_n_proteins
three_sqrt_qualifying_proteins = top_n_proteins(sqrt_qualifying_proteins, n=3)
three_sqrt_query_filepath = directory + "three_prot_clusters.txt"
print_querylist_of_clusters_to_file(clusters, clusters.get_all_cluster_labels(), three_sqrt_query_filepath, three_sqrt_qualifying_proteins)
sqrt_three_fe = get_initialized_fe(background_proteins_filepath, term_mapping_filepath, termlist = background_go_terms)
sqrt_three_fe.importFiles({'query': three_sqrt_query_filepath })
sqrt_three_fe.run(cluster=False)
print(f"number of functionally enriched clusters after addition of 3 proteins: {sqrt_three_fe.enrichment['Module'].nunique()}")


number of functionally enriched clusters after addition of 3 proteins: 483


In [25]:
half_qualifying_clusters, half_qualifying_proteins = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=0, cluster_constant=2, use_sqrt=False, protein_ratio=.50, protein_constant=0)
quarter_qualifying_clusters, quarter_qualifying_proteins = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=0, cluster_constant=2, use_sqrt=False, protein_ratio=.25, protein_constant=0)
tenth_qualifying_clusters, tenth_qualifying_proteins = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=0, cluster_constant=2, use_sqrt=False, protein_ratio=.10, protein_constant=0)


In [26]:
half_query_filepath = directory + "half_clusters.txt"
print_querylist_of_clusters_to_file(clusters, clusters.get_all_cluster_labels(), half_query_filepath, half_qualifying_proteins)
half_fe = get_initialized_fe(background_proteins_filepath, term_mapping_filepath, termlist = background_go_terms)
half_fe.importFiles({'query': half_query_filepath })
half_fe.run(cluster=False)

print(f"number of functionally enriched clusters after addition of 1/2 proteins: {half_fe.enrichment['Module'].nunique()}")


quarter_query_filepath = directory + "quarter_clusters.txt"
print_querylist_of_clusters_to_file(clusters, clusters.get_all_cluster_labels(), quarter_query_filepath, quarter_qualifying_proteins)
quarter_fe = get_initialized_fe(background_proteins_filepath, term_mapping_filepath, termlist = background_go_terms)
quarter_fe.importFiles({'query': quarter_query_filepath })
quarter_fe.run(cluster=False)

print(f"number of functionally enriched clusters after addition of 1/4 proteins: {quarter_fe.enrichment['Module'].nunique()}")


tenth_query_filepath = directory + "tenth_clusters.txt"
print_querylist_of_clusters_to_file(clusters, clusters.get_all_cluster_labels(), tenth_query_filepath, tenth_qualifying_proteins)
tenth_fe = get_initialized_fe(background_proteins_filepath, term_mapping_filepath, termlist = background_go_terms)
tenth_fe.importFiles({'query': tenth_query_filepath })
tenth_fe.run(cluster=False)

print(f"number of functionally enriched clusters after addition of 1/10 proteins: {tenth_fe.enrichment['Module'].nunique()}")


print(half_qualifying_proteins)
print(quarter_qualifying_proteins)  
print(tenth_qualifying_proteins)


number of functionally enriched clusters after addition of 1/2 proteins: 483
number of functionally enriched clusters after addition of 1/4 proteins: 482
number of functionally enriched clusters after addition of 1/10 proteins: 530
{}
{7: ['SSBP1'], 101: ['WDR64', 'PSMD8', 'PSMD2'], 117: ['ANAPC5', 'SMAD2'], 163: ['POLR3K'], 227: ['KXD1', 'RPL34'], 263: ['CTSD'], 317: ['MPP6'], 337: ['CCNK'], 356: ['PAX4'], 431: ['WNT4'], 467: ['CHD5'], 501: ['STAT1', 'NOS3'], 510: ['PLCB4', 'PLCB2', 'PLCB3', 'PLCB1'], 523: ['SLC30A10']}
{0: ['DPH2', 'DPH1', 'VDR', 'LSM3', 'TMEM30B', 'PLS3', 'PLS1', 'ESR2', 'AAMP', 'LCP1', 'GPN1', 'COPG2', 'PPARA', 'PPARG', 'FAM203B', 'WDR33', 'FAM203A', 'CNNM3', 'COPG1', 'DNAJB11', 'CNNM4', 'CNNM2', 'KDM5B', 'TGFB1', 'CNNM1', 'POLR3H', 'WNT10B', 'EXOSC6', 'NOL6', 'TWISTNB', 'PSMA3', 'SSBP1', 'EXOSC5', 'WDFY2', 'EXOSC4'], 2: ['PARP4', 'PARP2', 'FUS', 'PARP1'], 3: ['VPS35'], 4: ['SAP30BP', 'SPC25', 'THUMPD1', 'MSI1', 'MSI2', 'KHDRBS3', 'DGCR14', 'GSG2', 'CENPK', 'KHDRBS